# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 8 2022 2:32PM,252664,16,SHL-8877024,"SHL Pharma, LLC",Released
1,Dec 8 2022 2:28PM,252663,21,418543359,"NBTY Global, Inc.",Released
2,Dec 8 2022 2:26PM,252662,19,ADV80008902,"AdvaGen Pharma, Ltd",Released
3,Dec 8 2022 2:26PM,252662,19,ADV80008903,"AdvaGen Pharma, Ltd",Released
4,Dec 8 2022 2:26PM,252662,19,ADV80008904,"AdvaGen Pharma, Ltd",Released
5,Dec 8 2022 2:26PM,252662,19,ADV80008905,"AdvaGen Pharma, Ltd",Released
6,Dec 8 2022 2:26PM,252662,19,ADV80008906,"AdvaGen Pharma, Ltd",Released
7,Dec 8 2022 2:26PM,252662,19,ADV80008912,"AdvaGen Pharma, Ltd",Released
8,Dec 8 2022 2:23PM,252653,20,ATR-I000031,"HVL, LLC dba Atrium Innovations",Released
9,Dec 8 2022 2:23PM,252661,21,418543343,"NBTY Global, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
65,252660,Released,1
66,252661,Released,1
67,252662,Released,6
68,252663,Released,1
69,252664,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
252660,NaN,NaN,NaN,1.0
252661,NaN,NaN,NaN,1.0
252662,NaN,NaN,NaN,6.0
252663,NaN,NaN,NaN,1.0
252664,NaN,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
252543,0.0,1.0,0.0,3.0
252545,0.0,0.0,0.0,2.0
252555,0.0,0.0,1.0,1.0
252558,0.0,0.0,0.0,3.0
252560,0.0,30.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
252543,0.0,1,0,3
252545,0.0,0,0,2
252555,0.0,0,1,1
252558,0.0,0,0,3
252560,0.0,30,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,252543,0.0,1,0,3
1,252545,0.0,0,0,2
2,252555,0.0,0,1,1
3,252558,0.0,0,0,3
4,252560,0.0,30,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,252543,0.0,1,,3
1,252545,0.0,,,2
2,252555,0.0,,1,1
3,252558,0.0,,,3
4,252560,0.0,30,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 8 2022 2:32PM,252664,16,"SHL Pharma, LLC"
1,Dec 8 2022 2:28PM,252663,21,"NBTY Global, Inc."
2,Dec 8 2022 2:26PM,252662,19,"AdvaGen Pharma, Ltd"
8,Dec 8 2022 2:23PM,252653,20,"HVL, LLC dba Atrium Innovations"
9,Dec 8 2022 2:23PM,252661,21,"NBTY Global, Inc."
10,Dec 8 2022 2:23PM,252660,16,Sartorius Bioprocess Solutions
11,Dec 8 2022 2:21PM,252659,21,"NBTY Global, Inc."
12,Dec 8 2022 2:19PM,252658,21,"NBTY Global, Inc."
13,Dec 8 2022 2:18PM,252657,21,"NBTY Global, Inc."
14,Dec 8 2022 2:17PM,252656,19,"Dr. Raju's Pharma Solutions, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Dec 8 2022 2:32PM,252664,16,"SHL Pharma, LLC",0.0,,,1
1,Dec 8 2022 2:28PM,252663,21,"NBTY Global, Inc.",0.0,,,1
2,Dec 8 2022 2:26PM,252662,19,"AdvaGen Pharma, Ltd",0.0,,,6
3,Dec 8 2022 2:23PM,252653,20,"HVL, LLC dba Atrium Innovations",0.0,,,1
4,Dec 8 2022 2:23PM,252661,21,"NBTY Global, Inc.",0.0,,,1
5,Dec 8 2022 2:23PM,252660,16,Sartorius Bioprocess Solutions,0.0,,,1
6,Dec 8 2022 2:21PM,252659,21,"NBTY Global, Inc.",0.0,,,1
7,Dec 8 2022 2:19PM,252658,21,"NBTY Global, Inc.",0.0,,,1
8,Dec 8 2022 2:18PM,252657,21,"NBTY Global, Inc.",0.0,,,1
9,Dec 8 2022 2:17PM,252656,19,"Dr. Raju's Pharma Solutions, LLC",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 2:32PM,252664,16,"SHL Pharma, LLC",1,,
1,Dec 8 2022 2:28PM,252663,21,"NBTY Global, Inc.",1,,
2,Dec 8 2022 2:26PM,252662,19,"AdvaGen Pharma, Ltd",6,,
3,Dec 8 2022 2:23PM,252653,20,"HVL, LLC dba Atrium Innovations",1,,
4,Dec 8 2022 2:23PM,252661,21,"NBTY Global, Inc.",1,,
5,Dec 8 2022 2:23PM,252660,16,Sartorius Bioprocess Solutions,1,,
6,Dec 8 2022 2:21PM,252659,21,"NBTY Global, Inc.",1,,
7,Dec 8 2022 2:19PM,252658,21,"NBTY Global, Inc.",1,,
8,Dec 8 2022 2:18PM,252657,21,"NBTY Global, Inc.",1,,
9,Dec 8 2022 2:17PM,252656,19,"Dr. Raju's Pharma Solutions, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 2:32PM,252664,16,"SHL Pharma, LLC",1,,
1,Dec 8 2022 2:28PM,252663,21,"NBTY Global, Inc.",1,,
2,Dec 8 2022 2:26PM,252662,19,"AdvaGen Pharma, Ltd",6,,
3,Dec 8 2022 2:23PM,252653,20,"HVL, LLC dba Atrium Innovations",1,,
4,Dec 8 2022 2:23PM,252661,21,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 2:32PM,252664,16,"SHL Pharma, LLC",1.0,NaN,NaN
1,Dec 8 2022 2:28PM,252663,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Dec 8 2022 2:26PM,252662,19,"AdvaGen Pharma, Ltd",6.0,NaN,NaN
3,Dec 8 2022 2:23PM,252653,20,"HVL, LLC dba Atrium Innovations",1.0,NaN,NaN
4,Dec 8 2022 2:23PM,252661,21,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 8 2022 2:32PM,252664,16,"SHL Pharma, LLC",1.0,0.0,0.0
1,Dec 8 2022 2:28PM,252663,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Dec 8 2022 2:26PM,252662,19,"AdvaGen Pharma, Ltd",6.0,0.0,0.0
3,Dec 8 2022 2:23PM,252653,20,"HVL, LLC dba Atrium Innovations",1.0,0.0,0.0
4,Dec 8 2022 2:23PM,252661,21,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5052064,142.0,2.0,2.0
12,252587,0.0,1.0,0.0
15,757766,2.0,9.0,74.0
16,1010483,4.0,0.0,0.0
19,1515784,18.0,23.0,0.0
20,757825,3.0,0.0,2.0
21,6063114,24.0,0.0,0.0
22,252583,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5052064,142.0,2.0,2.0
1,12,252587,0.0,1.0,0.0
2,15,757766,2.0,9.0,74.0
3,16,1010483,4.0,0.0,0.0
4,19,1515784,18.0,23.0,0.0
5,20,757825,3.0,0.0,2.0
6,21,6063114,24.0,0.0,0.0
7,22,252583,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,142.0,2.0,2.0
1,12,0.0,1.0,0.0
2,15,2.0,9.0,74.0
3,16,4.0,0.0,0.0
4,19,18.0,23.0,0.0
5,20,3.0,0.0,2.0
6,21,24.0,0.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,142.0
1,12,Released,0.0
2,15,Released,2.0
3,16,Released,4.0
4,19,Released,18.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,2.0,0.0,74.0,0.0,0.0,2.0,0.0,0.0
Executing,2.0,1.0,9.0,0.0,23.0,0.0,0.0,0.0
Released,142.0,0.0,2.0,4.0,18.0,3.0,24.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,2.0,0.0,74.0,0.0,0.0,2.0,0.0,0.0
1,Executing,2.0,1.0,9.0,0.0,23.0,0.0,0.0,0.0
2,Released,142.0,0.0,2.0,4.0,18.0,3.0,24.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,2.0,0.0,74.0,0.0,0.0,2.0,0.0,0.0
1,Executing,2.0,1.0,9.0,0.0,23.0,0.0,0.0,0.0
2,Released,142.0,0.0,2.0,4.0,18.0,3.0,24.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()